In [1]:
%%time

import os
import pandas as pd
import numpy as np
import scipy.optimize as optimize
#data reading and cleaning
import gc
from tqdm import tqdm
items = pd.read_csv('Data/items.csv')
samples = pd.read_csv('Data/samples.csv')
test = pd.read_csv('Data/test.csv')
train = pd.read_csv('Data/train.csv')
users = pd.read_csv('Data/users.csv')


train_cols ={
    'User ID':'usr_id',
    'Item1 ID':'item1_id',
    'Item2 ID':'item2_id',
    'Preference':'preference'
}
train = train.rename(columns=train_cols)

test_cols ={
    'User ID':'usr_id',
    'Item1 ID':'item1_id',
    'Item2 ID':'item2_id',
}
test = test.rename(columns=test_cols)

users_cols ={
    'User ID':'usr_id',
    ' Education':'education',
    ' Age':'age',
    ' Gender':'gender',
    ' Region':'region'
}
users = users.rename(columns=users_cols)

# data merging

item1_cols ={
    'Item ID':'item1_id',
    ' BodyType':'body_type_1',
    ' Transmission':'transmission_1',
    ' Engin Capacity':'engin_capacity_1',
    ' Fuel Consumed':'fuel_consumed_1'
}

item2_cols ={
    'Item ID':'item2_id',
    ' BodyType':'body_type_2',
    ' Transmission':'transmission_2',
    ' Engin Capacity':'engin_capacity_2',
    ' Fuel Consumed':'fuel_consumed_2'
}

def find_winner():
    k = pd.read_csv('Data/train.csv')
    train_cols ={
        'User ID':'usr_id',
        'Item1 ID':'item1_id',
        'Item2 ID':'item2_id',
        'Preference':'preference'
    }
    k = k.rename(columns=train_cols)

    win=list()
    for index in k.index:
        if k.preference[index] ==0:
            win.append(k.item1_id[index])
        else:
            win.append(k.item2_id[index])
    k['win'] = win
    
    return k

def number_count(k):
    k = k.drop(columns=['preference','win'])

    p1 = k.pivot_table(index = ['usr_id'],columns=['item1_id'], aggfunc='count').fillna(0)
    p2= k.pivot_table(index = ['usr_id'],columns=['item2_id'], aggfunc='count').fillna(0)

    p1.columns = p1.columns.to_series().str.join('_')
    p2.columns = p2.columns.to_series().str.join('_')

    p1.columns = ['1_count','2_count','3_count','4_count','5_count',
                 '6_count','7_count','8_count','9_count','10_count']

    p2.columns = ['1_count','2_count','3_count','4_count','5_count',
                 '6_count','7_count','8_count','9_count','10_count']
    del p1.index.name, p2.index.name
    return p1+p2

def winner_count(k):
    k = k.drop(columns=['item1_id','item2_id'])

    result = k.pivot_table(index = ['usr_id'],columns=['win'], aggfunc='count')

    result.columns.to_series().str.join('_')
    del result.index.name

    #result.columns = result.columns.get_level_values(0)
    result.columns = ['1_count','2_count','3_count','4_count','5_count',
                 '6_count','7_count','8_count','9_count','10_count']
    return result

def occur_rate(k):
    k = (k/9).fillna(0)
    k['usr_id'] = k.index
    
    return k

def win_rate(w_count,n_count):
    k = (w_count/n_count).fillna(0)
    k.columns = ['1_win','2_win','3_win','4_win','5_win',
                '6_win','7_win','8_win','9_win','10_win']
    k['usr_id'] = k.index
    
    return k
    
def max_f(params):
    r1,r2,r3,r4,r5,r6,r7,r8,r9,r10 = params
    R = [r1,r2,r3,r4,r5,r6,r7,r8,r9,r10]
    index:int
    result = 1
    for index in data.index:
        result *= R[data.win[index]-1] / (R[data.item1_id[index]-1] + R[data.item2_id[index]-1])
        
    return -result


#calculate occur times and win rate

winner_list = find_winner()
n_count = number_count(winner_list)
w_count = winner_count(winner_list)
occur = occur_rate(n_count)
win = win_rate(w_count,n_count)

#concat train and test
df = pd.concat([train,test])
item_1 = items.copy().rename(columns = item1_cols)
item_2 = items.copy().rename(columns = item2_cols)
df = pd.merge(df,users, on =['usr_id'])
df = pd.merge(df,item_1, on =['item1_id'])
df = pd.merge(df,item_2, on =['item2_id'])
df.shape

del item_1,item_2
gc.collect()
#feature generation
#df = pd.merge(df,occur,on = ['usr_id'])
df = pd.merge(df,win,on = ['usr_id'])
df['engin2_bigger'] = 0
df.engin2_bigger[df.engin_capacity_1==df.engin_capacity_2] = 2
df.engin2_bigger[df.engin_capacity_1>df.engin_capacity_2] = 0
df.engin2_bigger[df.engin_capacity_1<df.engin_capacity_2] = 1

del n_count,w_count,occur,win
gc.collect()

#calculate likelyhood

r_cobyla_list = list()
r_powell_list = list()
initial_guess = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
for i in tqdm(range(60)):
    data = winner_list[winner_list['usr_id']==i]
    result_cobyla = optimize.minimize(max_f, initial_guess, method='COBYLA')
    result_powell = optimize.minimize(max_f, initial_guess, method='Powell')
    r_cobyla_list.append(result_cobyla.x - result_cobyla.x.min() + 1)
    r_powell_list.append(result_powell.x - result_powell.x.min()+1)
    

powell = list()
cobylab = list()
for index in tqdm(df.index):
    R_powell = r_powell_list[df.usr_id[index]-1]
    R_cobylab = r_cobyla_list[df.usr_id[index]-1]
    prob_p = R_powell[df.item2_id[index]-1] / (R_powell[df.item1_id[index]-1] + R_powell[df.item2_id[index]-1])
    prob_c = R_powell[df.item2_id[index]-1] / (R_powell[df.item1_id[index]-1] + R_powell[df.item2_id[index]-1])
    
    powell.append(prob_p)
    cobylab.append(prob_c)
    

df['powell'] = powell
#df['cobylab'] = cobylab

del powell#, cobylab
gc.collect()
#split train test
test = df[df.preference.isna()]
train = df[df.preference.isna()==0]

label = train.preference

train = train.drop(columns=['preference'])
test = test.drop(columns=['preference'])

#train = train.drop(columns=['preference','usr_id','item1_id','item2_id'])
#test = test.drop(columns=['preference','usr_id','item1_id','item2_id'])
#train_test split

from sklearn.model_selection import train_test_split
train_data, valid_data, train_target, valid_target = train_test_split(train, label, test_size=0.33, random_state=9)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 13%|█▎        | 8/60 [00:37<04:03,  4.67s/it]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:128: RuntimeWarning: divide by zero encountered in do

CPU times: user 4min 45s, sys: 1.58 s, total: 4min 46s
Wall time: 5min 10s


In [3]:
train_data.head()

,item1_id,item2_id,usr_id,education,age,gender,region,body_type_1,transmission_1,engin_capacity_1,...,3_win,4_win,5_win,6_win,7_win,8_win,9_win,10_win,engin2_bigger,powell
881,7,6,36,3,2,1,2,1,2,3.5,...,1.000000,1.000000,0.8,0.00,0.285714,0.00,1.000000,0.5,2,0.500000
320,9,8,48,3,2,2,1,1,2,3.5,...,1.000000,0.500000,0.5,0.00,0.571429,0.00,1.000000,0.5,0,0.750886
1182,1,5,40,3,1,1,1,2,1,2.5,...,0.833333,0.666667,0.8,0.00,0.285714,0.25,1.000000,0.5,1,0.500000
1938,7,3,51,2,3,1,1,1,2,3.5,...,1.000000,0.333333,0.6,0.25,0.428571,0.75,0.666667,0.0,1,1.000000
1747,3,8,57,3,3,2,1,1,1,4.5,...,0.500000,0.000000,0.0,0.75,1.000000,0.80,0.500000,0.0,0,0.250947


In [ ]:
#train_data, valid_data, train_target, valid_target

from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp, rand
import lightgbm as lgb

'''d_train = lgb.Dataset(train_data, train_target)
d_valid = lgb.Dataset(valid_data, valid_target)'''

parameter_space_lgb ={
    'objective' :'binary',
    'metric': 'binary_logloss',
    'boost' : hp.choice('boost',['gbdt', 'rf', 'dart', 'goss']),
    'learning_rate':hp.uniform('learning_rate',0.01,0.2),
    'num_leaves':hp.randint('num_leaves',100)+2,
    'tree_learner':hp.choice('tree_learner',['serial', 'feature', 'data', 'voting']),
    'num_threads':2,
    'min_data_in_leaf':hp.randint('min_data_in_leaf',80)+1,
    'feature_fraction':hp.uniform('feature_fraction', 0.2, 0.8),
    'bagging_fraction':hp.uniform('bagging_fraction', 0.2, 0.8),
    'bagging_freq': hp.randint('bagging_freq',4)+1
    
}

count = 0

def function(args):
    print(args)

    # **可以把dict转换为关键字参数，可以大大简化复杂的函数调用
    clf = lgb.LGBMClassifier(**args )
    clf.fit(train_data,train_target)
    # 预测测试集
    prediction = clf.predict(valid_data)

    global count
    count = count + 1
    score = accuracy_score(valid_target, prediction)
    print("第%s次，测试集正确率为：" % str(count),score)

    # 由于hyperopt仅提供fmin接口，因此如果要求最大值，则需要取相反数
    return -score

# algo指定搜索算法，目前支持以下算法：
# ①随机搜索(hyperopt.rand.suggest)
# ②模拟退火(hyperopt.anneal.suggest)
# ③TPE算法（hyperopt.tpe.suggest，算法全称为Tree-structured Parzen Estimator Approach）
# max_evals指定枚举次数上限，即使第max_evals次枚举仍未能确定全局最优解，也要结束搜索，返回目前搜索到的最优解

best = fmin(function, parameter_space_lgb, algo=tpe.suggest, max_evals=10000)

# best["kernel"]返回的是数组下标，因此需要把它还原回来
boost_list = ['gbdt', 'rf', 'dart', 'goss']
best["boost"] = boost_list[best["boost"]]

tree_learner_list = ['serial', 'feature', 'data', 'voting']
best["tree_learner"] = tree_learner_list[best["tree_learner"]]

print("最佳参数为：",best)

In [10]:
train_data.columns

Index(['item1_id', 'item2_id', 'usr_id', 'education', 'age', 'gender',
       'region', 'body_type_1', 'transmission_1', 'engin_capacity_1',
       'fuel_consumed_1', 'body_type_2', 'transmission_2', 'engin_capacity_2',
       'fuel_consumed_2', '1_win', '2_win', '3_win', '4_win', '5_win', '6_win',
       '7_win', '8_win', '9_win', '10_win', 'engin2_bigger', 'powell',
       'cobylab'],
      dtype='object')

In [12]:
import lightgbm as lgb

#test = test.drop(columns=['User-Item1-Item2','Preference'])
clf = lgb.LGBMClassifier(bagging_fraction= 0.6409803582029194, bagging_freq= 2, boost= 'dart',
                         feature_fraction= 0.7076381520129653, learning_rate= 0.09672900472668634,
                         min_data_in_leaf= 27, num_leaves= 74, tree_learner= 'serial')
clf.fit(train,label)
# 预测测试集
prediction = clf.predict(test)

test["User-Item1-Item2"] = test['usr_id'].map(str)+'-'+test['item1_id'].map(str)+'-'+test['item2_id'].map(str)
test['Preference'] = prediction.astype(np.int8)

submission = test[['User-Item1-Item2','Preference']]

submission.to_csv('submission9_Lgb.csv',index = False)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [30]:
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp, rand
from sklearn.linear_model import LogisticRegression   

parameter_space_logit ={
    # loguniform表示该参数取对数后符合均匀分布
    'C':hp.loguniform("C", np.log(1), np.log(100)),
    #'dual':hp.choice('dual',[1,0]),
    'penalty':hp.choice('penalty',['l1','l2']),
}

count = 0

def function(args):
    print(args)

    # **可以把dict转换为关键字参数，可以大大简化复杂的函数调用
    clf = LogisticRegression(**args)

    # 训练模型
    clf.fit(train_data,train_target)

    # 预测测试集
    prediction = clf.predict(valid_data)

    global count
    count = count + 1
    score = accuracy_score(valid_target, prediction)
    print("第%s次，测试集正确率为：" % str(count),score)

    # 由于hyperopt仅提供fmin接口，因此如果要求最大值，则需要取相反数
    return -score

# algo指定搜索算法，目前支持以下算法：
# ①随机搜索(hyperopt.rand.suggest)
# ②模拟退火(hyperopt.anneal.suggest)
# ③TPE算法（hyperopt.tpe.suggest，算法全称为Tree-structured Parzen Estimator Approach）
# max_evals指定枚举次数上限，即使第max_evals次枚举仍未能确定全局最优解，也要结束搜索，返回目前搜索到的最优解
best = fmin(function, parameter_space_logit, algo=tpe.suggest, max_evals=10000)

# best["kernel"]返回的是数组下标，因此需要把它还原回来
penalty_list = ['l1','l2']
best["penalty"] = penalty_list[best["penalty"]]

print("最佳参数为：",best)


{'C': 2.5840642684230892, 'penalty': 'l1'}
第1次，测试集正确率为： 0.623059866962306
{'C': 17.426548443056593, 'penalty': 'l2'}
第2次，测试集正确率为： 0.6164079822616408
{'C': 2.8538305061080806, 'penalty': 'l1'}
第3次，测试集正确率为： 0.6252771618625277
{'C': 10.451344124370221, 'penalty': 'l2'}
第4次，测试集正确率为： 0.6208425720620843
{'C': 41.29943538957868, 'penalty': 'l1'}
第5次，测试集正确率为： 0.6075388026607539
{'C': 36.721460057032296, 'penalty': 'l1'}
第6次，测试集正确率为： 0.6075388026607539
{'C': 1.4084402294040337, 'penalty': 'l1'}
第7次，测试集正确率为： 0.623059866962306
{'C': 80.3783071214435, 'penalty': 'l2'}
第8次，测试集正确率为： 0.6097560975609756
{'C': 30.281564975211953, 'penalty': 'l2'}
第9次，测试集正确率为： 0.6119733924611973
{'C': 22.64344873469269, 'penalty': 'l1'}
第10次，测试集正确率为： 0.6097560975609756
{'C': 9.028802882383125, 'penalty': 'l2'}
第11次，测试集正确率为： 0.623059866962306
{'C': 4.615169055506717, 'penalty': 'l1'}
第12次，测试集正确率为： 0.623059866962306
{'C': 74.49921944744187, 'penalty': 'l2'}
第13次，测试集正确率为： 0.6097560975609756
{'C': 1.0294592266978524, 'penal

{'C': 6.241813784318241, 'penalty': 'l2'}
第112次，测试集正确率为： 0.6252771618625277
{'C': 1.8526512173728908, 'penalty': 'l1'}
第113次，测试集正确率为： 0.6274944567627494
{'C': 3.477617126380365, 'penalty': 'l1'}
第114次，测试集正确率为： 0.6252771618625277
{'C': 1.7261082545934632, 'penalty': 'l1'}
第115次，测试集正确率为： 0.6274944567627494
{'C': 44.099172294459116, 'penalty': 'l1'}
第116次，测试集正确率为： 0.6075388026607539
{'C': 1.99920871206902, 'penalty': 'l2'}
第117次，测试集正确率为： 0.623059866962306
{'C': 27.994462758926634, 'penalty': 'l1'}
第118次，测试集正确率为： 0.6097560975609756
{'C': 1.3679015943023318, 'penalty': 'l1'}
第119次，测试集正确率为： 0.623059866962306
{'C': 1.007073239957625, 'penalty': 'l1'}
第120次，测试集正确率为： 0.6274944567627494
{'C': 2.371484446358162, 'penalty': 'l1'}
第121次，测试集正确率为： 0.6252771618625277
{'C': 2.753685313309976, 'penalty': 'l2'}
第122次，测试集正确率为： 0.6252771618625277
{'C': 11.111166281559242, 'penalty': 'l1'}
第123次，测试集正确率为： 0.6164079822616408
{'C': 8.626472580294777, 'penalty': 'l1'}
第124次，测试集正确率为： 0.6186252771618626
{'C': 1.4

第220次，测试集正确率为： 0.6164079822616408
{'C': 4.844793821597986, 'penalty': 'l1'}
第221次，测试集正确率为： 0.623059866962306
{'C': 9.967117376014917, 'penalty': 'l1'}
第222次，测试集正确率为： 0.6164079822616408
{'C': 1.72064966881021, 'penalty': 'l2'}
第223次，测试集正确率为： 0.6252771618625277
{'C': 12.63730297619479, 'penalty': 'l1'}
第224次，测试集正确率为： 0.6164079822616408
{'C': 3.974624836188734, 'penalty': 'l1'}
第225次，测试集正确率为： 0.6252771618625277
{'C': 1.4422303347028866, 'penalty': 'l1'}
第226次，测试集正确率为： 0.623059866962306
{'C': 18.978045575509668, 'penalty': 'l1'}
第227次，测试集正确率为： 0.6119733924611973
{'C': 1.1311352653443802, 'penalty': 'l2'}
第228次，测试集正确率为： 0.623059866962306
{'C': 6.2047764337133575, 'penalty': 'l1'}
第229次，测试集正确率为： 0.6274944567627494
{'C': 89.7096963465177, 'penalty': 'l1'}
第230次，测试集正确率为： 0.6075388026607539
{'C': 2.1856786509204356, 'penalty': 'l1'}
第231次，测试集正确率为： 0.6252771618625277
{'C': 3.2927820201914546, 'penalty': 'l1'}
第232次，测试集正确率为： 0.6252771618625277
{'C': 7.543869552983982, 'penalty': 'l1'}
第233次，测试集正确

第328次，测试集正确率为： 0.6164079822616408
{'C': 3.3936512875079954, 'penalty': 'l1'}
第329次，测试集正确率为： 0.6252771618625277
{'C': 5.908837251733926, 'penalty': 'l1'}
第330次，测试集正确率为： 0.6252771618625277
{'C': 4.8176117668089, 'penalty': 'l1'}
第331次，测试集正确率为： 0.623059866962306
{'C': 3.052879471497968, 'penalty': 'l2'}
第332次，测试集正确率为： 0.623059866962306
{'C': 1.0186460131480901, 'penalty': 'l1'}
第333次，测试集正确率为： 0.6252771618625277
{'C': 6.626975749045503, 'penalty': 'l1'}
第334次，测试集正确率为： 0.6274944567627494
{'C': 5.739741607310869, 'penalty': 'l1'}
第335次，测试集正确率为： 0.6274944567627494
{'C': 4.029055046963974, 'penalty': 'l1'}
第336次，测试集正确率为： 0.623059866962306
{'C': 8.419658703433585, 'penalty': 'l1'}
第337次，测试集正确率为： 0.6164079822616408
{'C': 8.919272528706266, 'penalty': 'l1'}
第338次，测试集正确率为： 0.6186252771618626
{'C': 5.438895851377444, 'penalty': 'l1'}
第339次，测试集正确率为： 0.6252771618625277
{'C': 5.179522036978987, 'penalty': 'l1'}
第340次，测试集正确率为： 0.6252771618625277
{'C': 41.79366828966336, 'penalty': 'l2'}
第341次，测试集正确率为： 

第437次，测试集正确率为： 0.6164079822616408
{'C': 1.8110540179148433, 'penalty': 'l1'}
第438次，测试集正确率为： 0.6252771618625277
{'C': 6.07446112044696, 'penalty': 'l1'}
第439次，测试集正确率为： 0.6274944567627494
{'C': 7.821428484572443, 'penalty': 'l1'}
第440次，测试集正确率为： 0.6164079822616408
{'C': 10.629286437801207, 'penalty': 'l2'}
第441次，测试集正确率为： 0.6208425720620843
{'C': 9.215914961954127, 'penalty': 'l1'}
第442次，测试集正确率为： 0.6186252771618626
{'C': 1.0091629941866942, 'penalty': 'l1'}
第443次，测试集正确率为： 0.6274944567627494
{'C': 45.97527494523918, 'penalty': 'l1'}
第444次，测试集正确率为： 0.6075388026607539
{'C': 5.184513706341965, 'penalty': 'l1'}
第445次，测试集正确率为： 0.6252771618625277
{'C': 1.146331585020704, 'penalty': 'l1'}
第446次，测试集正确率为： 0.6252771618625277
{'C': 2.4584048824964864, 'penalty': 'l2'}
第447次，测试集正确率为： 0.6252771618625277
{'C': 6.541806350592238, 'penalty': 'l1'}
第448次，测试集正确率为： 0.6274944567627494
{'C': 4.5571067424845495, 'penalty': 'l1'}
第449次，测试集正确率为： 0.623059866962306
{'C': 4.1107599120434735, 'penalty': 'l1'}
第450次，测试

...

{'C': 5.031617071182247, 'penalty': 'l1'}
第9483次，测试集正确率为： 0.623059866962306
{'C': 2.9079223427714957, 'penalty': 'l1'}
第9484次，测试集正确率为： 0.6252771618625277
{'C': 6.237349139434205, 'penalty': 'l1'}
第9485次，测试集正确率为： 0.6274944567627494
{'C': 6.60716103545518, 'penalty': 'l1'}
第9486次，测试集正确率为： 0.6274944567627494
{'C': 6.063437438810864, 'penalty': 'l1'}
第9487次，测试集正确率为： 0.6274944567627494
{'C': 4.6643689424006745, 'penalty': 'l1'}
第9488次，测试集正确率为： 0.623059866962306
{'C': 1.9900816383706559, 'penalty': 'l1'}
第9489次，测试集正确率为： 0.6252771618625277
{'C': 4.750940610245678, 'penalty': 'l1'}
第9490次，测试集正确率为： 0.623059866962306
{'C': 4.233614750199253, 'penalty': 'l1'}
第9491次，测试集正确率为： 0.623059866962306
{'C': 3.9783727359249736, 'penalty': 'l1'}
第9492次，测试集正确率为： 0.623059866962306
{'C': 3.494238773249392, 'penalty': 'l1'}
第9493次，测试集正确率为： 0.6252771618625277
{'C': 12.544207717707582, 'penalty': 'l1'}
第9494次，测试集正确率为： 0.614190687361419
{'C': 5.598205870106148, 'penalty': 'l1'}
第9495次，测试集正确率为： 0.6274944567627494
{

{'C': 6.457387987457304, 'penalty': 'l1'}
第9590次，测试集正确率为： 0.6274944567627494
{'C': 6.256428996462194, 'penalty': 'l1'}
第9591次，测试集正确率为： 0.6274944567627494
{'C': 6.03564961004697, 'penalty': 'l1'}
第9592次，测试集正确率为： 0.6274944567627494
{'C': 5.890682550896542, 'penalty': 'l1'}
第9593次，测试集正确率为： 0.6274944567627494
{'C': 4.539025644588938, 'penalty': 'l1'}
第9594次，测试集正确率为： 0.623059866962306
{'C': 4.606989165328174, 'penalty': 'l1'}
第9595次，测试集正确率为： 0.623059866962306
{'C': 5.604928216820252, 'penalty': 'l1'}
第9596次，测试集正确率为： 0.6274944567627494
{'C': 5.603446355987455, 'penalty': 'l1'}
第9597次，测试集正确率为： 0.6274944567627494
{'C': 5.3610775469223455, 'penalty': 'l1'}
第9598次，测试集正确率为： 0.6252771618625277
{'C': 5.190584537916226, 'penalty': 'l1'}
第9599次，测试集正确率为： 0.6252771618625277
{'C': 5.1161485537664735, 'penalty': 'l1'}
第9600次，测试集正确率为： 0.623059866962306
{'C': 5.788960102123388, 'penalty': 'l1'}
第9601次，测试集正确率为： 0.6274944567627494
{'C': 5.23899400040212, 'penalty': 'l1'}
第9602次，测试集正确率为： 0.6252771618625277
{'

{'C': 6.683916805041796, 'penalty': 'l2'}
第9697次，测试集正确率为： 0.623059866962306
{'C': 6.6632637585948755, 'penalty': 'l1'}
第9698次，测试集正确率为： 0.6274944567627494
{'C': 6.143740630854329, 'penalty': 'l1'}
第9699次，测试集正确率为： 0.6274944567627494
{'C': 5.498820064068617, 'penalty': 'l1'}
第9700次，测试集正确率为： 0.6252771618625277
{'C': 4.854020712456016, 'penalty': 'l1'}
第9701次，测试集正确率为： 0.623059866962306
{'C': 6.341098944075185, 'penalty': 'l1'}
第9702次，测试集正确率为： 0.6274944567627494
{'C': 4.897180037308141, 'penalty': 'l1'}
第9703次，测试集正确率为： 0.623059866962306
{'C': 5.982895917477404, 'penalty': 'l1'}
第9704次，测试集正确率为： 0.6252771618625277
{'C': 6.346675686635473, 'penalty': 'l1'}
第9705次，测试集正确率为： 0.6274944567627494
{'C': 6.148012255952799, 'penalty': 'l1'}
第9706次，测试集正确率为： 0.6274944567627494
{'C': 5.945022163180597, 'penalty': 'l1'}
第9707次，测试集正确率为： 0.6252771618625277
{'C': 5.928037524320271, 'penalty': 'l1'}
第9708次，测试集正确率为： 0.6252771618625277
{'C': 7.053797565315305, 'penalty': 'l1'}
第9709次，测试集正确率为： 0.623059866962306
{'

{'C': 1.77839603573153, 'penalty': 'l1'}
第9805次，测试集正确率为： 0.6252771618625277
{'C': 4.88614502362249, 'penalty': 'l1'}
第9806次，测试集正确率为： 0.623059866962306
{'C': 19.344108839053742, 'penalty': 'l1'}
第9807次，测试集正确率为： 0.6119733924611973
{'C': 6.169604919803174, 'penalty': 'l1'}
第9808次，测试集正确率为： 0.6274944567627494
{'C': 7.017602375828609, 'penalty': 'l1'}
第9809次，测试集正确率为： 0.6252771618625277
{'C': 5.878087388503595, 'penalty': 'l1'}
第9810次，测试集正确率为： 0.6274944567627494
{'C': 1.7905741762456278, 'penalty': 'l1'}
第9811次，测试集正确率为： 0.6252771618625277
{'C': 6.085293265087069, 'penalty': 'l1'}
第9812次，测试集正确率为： 0.6274944567627494
{'C': 8.540469290486758, 'penalty': 'l1'}
第9813次，测试集正确率为： 0.6164079822616408
{'C': 6.134306873986024, 'penalty': 'l1'}
第9814次，测试集正确率为： 0.6274944567627494
{'C': 4.9222317219110385, 'penalty': 'l1'}
第9815次，测试集正确率为： 0.623059866962306
{'C': 5.037137581074905, 'penalty': 'l1'}
第9816次，测试集正确率为： 0.6252771618625277
{'C': 8.072070430849942, 'penalty': 'l1'}
第9817次，测试集正确率为： 0.6164079822616408


{'C': 5.486215828159201, 'penalty': 'l1'}
第9912次，测试集正确率为： 0.6252771618625277
{'C': 5.487539001889627, 'penalty': 'l1'}
第9913次，测试集正确率为： 0.6252771618625277
{'C': 4.201753273939158, 'penalty': 'l1'}
第9914次，测试集正确率为： 0.623059866962306
{'C': 5.91387357588786, 'penalty': 'l1'}
第9915次，测试集正确率为： 0.6252771618625277
{'C': 5.437602522644465, 'penalty': 'l1'}
第9916次，测试集正确率为： 0.6252771618625277
{'C': 4.684493270177773, 'penalty': 'l1'}
第9917次，测试集正确率为： 0.623059866962306
{'C': 7.003171314357347, 'penalty': 'l1'}
第9918次，测试集正确率为： 0.6274944567627494
{'C': 4.454440943262022, 'penalty': 'l1'}
第9919次，测试集正确率为： 0.623059866962306
{'C': 5.085257958990852, 'penalty': 'l1'}
第9920次，测试集正确率为： 0.6252771618625277
{'C': 5.629268544506234, 'penalty': 'l1'}
第9921次，测试集正确率为： 0.6274944567627494
{'C': 5.600126679487972, 'penalty': 'l1'}
第9922次，测试集正确率为： 0.6274944567627494
{'C': 4.969747603204964, 'penalty': 'l1'}
第9923次，测试集正确率为： 0.623059866962306
{'C': 7.287976450947472, 'penalty': 'l1'}
第9924次，测试集正确率为： 0.623059866962306
{'C':

In [73]:
train_data.head()

,item1_id,item2_id,usr_id,education,age,gender,region,body_type_1,transmission_1,engin_capacity_1,fuel_consumed_1,body_type_2,transmission_2,engin_capacity_2,fuel_consumed_2,engin2_bigger,powell
735,3,8,36,3,2,1,2,1,1,4.5,2,2,2,2.5,1,0,0.500000
261,1,7,9,3,3,1,4,2,1,2.5,2,1,2,3.5,1,1,0.500000
1057,7,10,31,2,1,2,4,1,2,3.5,1,2,2,4.5,2,1,0.499573
1888,4,5,58,2,2,2,1,1,1,6.2,2,2,1,3.5,2,0,0.501042
1674,10,9,4,1,4,1,3,2,2,4.5,2,1,2,3.5,2,0,0.250904


In [ ]:
%%time
  
classifier = LogisticRegression(C= 20.402633905230882, penalty= 'l1')  # 使用类，参数全是默认的  
classifier.fit(train_data, train_target)  # 训练数据来学习，不需要返回值  
#print(accuracy_score(valid_target, classifier.predict(valid_data)))

prediction = classifier.predict(test)

test["User-Item1-Item2"] = test['usr_id'].map(str)+'-'+test['item1_id'].map(str)+'-'+test['item2_id'].map(str)
test['Preference'] = prediction.astype(np.int8)

submission = test[['User-Item1-Item2','Preference']]

submission.to_csv('submission3_Logit.csv',index = False)

In [14]:
#coding:utf-8
from hyperopt import fmin, tpe, hp, rand
from sklearn import svm


parameter_space_svc ={
    # loguniform表示该参数取对数后符合均匀分布
    'C':hp.loguniform("C", np.log(1), np.log(100)),
    'kernel':hp.choice('kernel',['rbf','poly']),
    'gamma': hp.loguniform("gamma", np.log(0.001), np.log(0.1)),
}


# 计数器，每一次参数组合的枚举都会使它加1
count = 0

def function(args):
    print(args)

    # **可以把dict转换为关键字参数，可以大大简化复杂的函数调用
    clf = svm.SVC(**args)

    # 训练模型
    clf.fit(train_data,train_target)

    # 预测测试集
    prediction = clf.predict(valid_data)

    global count
    count = count + 1
    score = accuracy_score(valid_target, prediction)
    print("第%s次，测试集正确率为：" % str(count),score)

    # 由于hyperopt仅提供fmin接口，因此如果要求最大值，则需要取相反数
    return -score

# algo指定搜索算法，目前支持以下算法：
# ①随机搜索(hyperopt.rand.suggest)
# ②模拟退火(hyperopt.anneal.suggest)
# ③TPE算法（hyperopt.tpe.suggest，算法全称为Tree-structured Parzen Estimator Approach）
# max_evals指定枚举次数上限，即使第max_evals次枚举仍未能确定全局最优解，也要结束搜索，返回目前搜索到的最优解
best = fmin(function, parameter_space_svc, algo=tpe.suggest, max_evals=10000)

# best["kernel"]返回的是数组下标，因此需要把它还原回来
kernel_list = ['rbf','poly']
best["kernel"] = kernel_list[best["kernel"]]

print("最佳参数为：",best)

clf = svm.SVC(**best)
print(clf)

{'C': 4.547494988685583, 'gamma': 0.0032771582095555353, 'kernel': 'poly'}


NameError: name 'f1_score' is not defined

In [10]:
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp, rand
from sklearn.neural_network import MLPClassifier 

parameter_space_NN = {'hidden_layer_sizes':hp.randint('hidden_layer_sizes', 99)+1,
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'solver':hp.choice('solver', ['lbfgs', 'sgd', 'adam']),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),'random_state':42
         #'learning_rate':hp.uniform('learning_rate', low=0.001, high=0.999),
        }

count = 0

def function(args):
    print(args)

    # **可以把dict转换为关键字参数，可以大大简化复杂的函数调用
    clf = MLPClassifier (**args)

    # 训练模型
    clf.fit(train_data,train_target)

    # 预测测试集
    prediction = clf.predict(valid_data)

    global count
    count = count + 1
    score = accuracy_score(valid_target, prediction)
    print("第%s次，测试集正确率为：" % str(count),score)

    # 由于hyperopt仅提供fmin接口，因此如果要求最大值，则需要取相反数
    return -score

# algo指定搜索算法，目前支持以下算法：
# ①随机搜索(hyperopt.rand.suggest)
# ②模拟退火(hyperopt.anneal.suggest)
# ③TPE算法（hyperopt.tpe.suggest，算法全称为Tree-structured Parzen Estimator Approach）
# max_evals指定枚举次数上限，即使第max_evals次枚举仍未能确定全局最优解，也要结束搜索，返回目前搜索到的最优解
best = fmin(function, parameter_space_NN, algo=tpe.suggest, max_evals=10000)

# best["kernel"]返回的是数组下标，因此需要把它还原回来
solver_list = ['lbfgs', 'sgd', 'adam']
best["solver"] = solver_list[best["solver"]]

activation_list = ['logistic', 'tanh', 'relu']
best["activation"] = activation_list[best["activation"]]

print("最佳参数为：",best)



{'activation': 'relu', 'alpha': 0.0001754303710409312, 'hidden_layer_sizes': 16, 'random_state': 42, 'solver': 'lbfgs'}
第1次，测试集正确率为： 0.6186252771618626
{'activation': 'logistic', 'alpha': 0.0001674473497793656, 'hidden_layer_sizes': 62, 'random_state': 42, 'solver': 'lbfgs'}
第2次，测试集正确率为： 0.7516629711751663
{'activation': 'logistic', 'alpha': 0.0005101769275369731, 'hidden_layer_sizes': 38, 'random_state': 42, 'solver': 'sgd'}
第3次，测试集正确率为： 0.49223946784922396
{'activation': 'logistic', 'alpha': 7.76469247655484e-05, 'hidden_layer_sizes': 31, 'random_state': 42, 'solver': 'adam'}


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


第4次，测试集正确率为： 0.656319290465632
{'activation': 'tanh', 'alpha': 0.00028115318743940663, 'hidden_layer_sizes': 17, 'random_state': 42, 'solver': 'sgd'}
第5次，测试集正确率为： 0.5343680709534369
{'activation': 'logistic', 'alpha': 3.793035993659845e-05, 'hidden_layer_sizes': 11, 'random_state': 42, 'solver': 'sgd'}
第6次，测试集正确率为： 0.4722838137472284
{'activation': 'relu', 'alpha': 0.00022103384098931845, 'hidden_layer_sizes': 85, 'random_state': 42, 'solver': 'sgd'}
第7次，测试集正确率为： 0.5720620842572062
{'activation': 'tanh', 'alpha': 0.00018201437894721514, 'hidden_layer_sizes': 4, 'random_state': 42, 'solver': 'sgd'}
第8次，测试集正确率为： 0.5144124168514412
{'activation': 'tanh', 'alpha': 9.290453822143957e-05, 'hidden_layer_sizes': 53, 'random_state': 42, 'solver': 'lbfgs'}
第9次，测试集正确率为： 0.7161862527716186
{'activation': 'relu', 'alpha': 0.00010057804987616232, 'hidden_layer_sizes': 56, 'random_state': 42, 'solver': 'sgd'}
第10次，测试集正确率为： 0.549889135254989
{'activation': 'relu', 'alpha': 0.00021165616459082785, 'hid

第58次，测试集正确率为： 0.6807095343680709
{'activation': 'relu', 'alpha': 0.0004012778818379242, 'hidden_layer_sizes': 50, 'random_state': 42, 'solver': 'adam'}
第59次，测试集正确率为： 0.623059866962306
{'activation': 'logistic', 'alpha': 0.0002120034957056817, 'hidden_layer_sizes': 46, 'random_state': 42, 'solver': 'lbfgs'}
第60次，测试集正确率为： 0.6762749445676275
{'activation': 'relu', 'alpha': 5.7963911157827035e-06, 'hidden_layer_sizes': 64, 'random_state': 42, 'solver': 'lbfgs'}
第61次，测试集正确率为： 0.6873614190687362
{'activation': 'tanh', 'alpha': 9.509092190759902e-05, 'hidden_layer_sizes': 39, 'random_state': 42, 'solver': 'sgd'}
第62次，测试集正确率为： 0.5432372505543237
{'activation': 'logistic', 'alpha': 2.8201057738501304e-05, 'hidden_layer_sizes': 78, 'random_state': 42, 'solver': 'lbfgs'}
第63次，测试集正确率为： 0.7161862527716186
{'activation': 'relu', 'alpha': 8.564712587902821e-06, 'hidden_layer_sizes': 97, 'random_state': 42, 'solver': 'lbfgs'}
第64次，测试集正确率为： 0.7472283813747228
{'activation': 'logistic', 'alpha': 2.01968

第112次，测试集正确率为： 0.738359201773836
{'activation': 'tanh', 'alpha': 2.9066290234860206e-05, 'hidden_layer_sizes': 31, 'random_state': 42, 'solver': 'adam'}
第113次，测试集正确率为： 0.738359201773836
{'activation': 'tanh', 'alpha': 3.45770173114047e-05, 'hidden_layer_sizes': 93, 'random_state': 42, 'solver': 'adam'}
第114次，测试集正确率为： 0.7982261640798226
{'activation': 'tanh', 'alpha': 3.480547361534909e-05, 'hidden_layer_sizes': 93, 'random_state': 42, 'solver': 'adam'}
第115次，测试集正确率为： 0.7982261640798226
{'activation': 'tanh', 'alpha': 3.478520376995726e-05, 'hidden_layer_sizes': 93, 'random_state': 42, 'solver': 'adam'}
第116次，测试集正确率为： 0.7982261640798226
{'activation': 'tanh', 'alpha': 4.3158460373198774e-05, 'hidden_layer_sizes': 56, 'random_state': 42, 'solver': 'adam'}
第117次，测试集正确率为： 0.7671840354767184
{'activation': 'tanh', 'alpha': 3.611609178779664e-05, 'hidden_layer_sizes': 12, 'random_state': 42, 'solver': 'adam'}
第118次，测试集正确率为： 0.6940133037694013
{'activation': 'tanh', 'alpha': 2.875938312958872

...

第9773次，测试集正确率为： 0.7849223946784922
{'activation': 'tanh', 'alpha': 0.00387624085333785, 'hidden_layer_sizes': 6, 'random_state': 42, 'solver': 'adam'}
第9774次，测试集正确率为： 0.6274944567627494
{'activation': 'relu', 'alpha': 0.0032701653465546953, 'hidden_layer_sizes': 30, 'random_state': 42, 'solver': 'adam'}
第9775次，测试集正确率为： 0.6053215077605322
{'activation': 'tanh', 'alpha': 0.0034123012905164078, 'hidden_layer_sizes': 22, 'random_state': 42, 'solver': 'adam'}
第9776次，测试集正确率为： 0.6696230598669624
{'activation': 'tanh', 'alpha': 0.0038513820734054406, 'hidden_layer_sizes': 70, 'random_state': 42, 'solver': 'adam'}
第9777次，测试集正确率为： 0.7782705099778271
{'activation': 'tanh', 'alpha': 0.003150521808260382, 'hidden_layer_sizes': 9, 'random_state': 42, 'solver': 'adam'}
第9778次，测试集正确率为： 0.6430155210643016
{'activation': 'tanh', 'alpha': 0.0035346740537744534, 'hidden_layer_sizes': 81, 'random_state': 42, 'solver': 'adam'}
第9779次，测试集正确率为： 0.7583148558758315
{'activation': 'tanh', 'alpha': 0.003400471245

第9827次，测试集正确率为： 0.7161862527716186
{'activation': 'tanh', 'alpha': 0.003689845891055392, 'hidden_layer_sizes': 69, 'random_state': 42, 'solver': 'adam'}
第9828次，测试集正确率为： 0.753880266075388
{'activation': 'tanh', 'alpha': 0.003356891463305355, 'hidden_layer_sizes': 51, 'random_state': 42, 'solver': 'adam'}
第9829次，测试集正确率为： 0.7760532150776053
{'activation': 'tanh', 'alpha': 0.0032341711027413147, 'hidden_layer_sizes': 83, 'random_state': 42, 'solver': 'adam'}
第9830次，测试集正确率为： 0.8203991130820399
{'activation': 'tanh', 'alpha': 0.0032676742607352805, 'hidden_layer_sizes': 29, 'random_state': 42, 'solver': 'adam'}
第9831次，测试集正确率为： 0.7139689578713969
{'activation': 'tanh', 'alpha': 0.0030888676799531926, 'hidden_layer_sizes': 66, 'random_state': 42, 'solver': 'adam'}
第9832次，测试集正确率为： 0.7583148558758315
{'activation': 'relu', 'alpha': 0.003065512525464216, 'hidden_layer_sizes': 12, 'random_state': 42, 'solver': 'adam'}
第9833次，测试集正确率为： 0.6585365853658537
{'activation': 'tanh', 'alpha': 0.00395916508

第9881次，测试集正确率为： 0.7361419068736141
{'activation': 'tanh', 'alpha': 0.0035615524265513133, 'hidden_layer_sizes': 50, 'random_state': 42, 'solver': 'adam'}
第9882次，测试集正确率为： 0.7804878048780488
{'activation': 'tanh', 'alpha': 0.0033829492725362263, 'hidden_layer_sizes': 99, 'random_state': 42, 'solver': 'adam'}
第9883次，测试集正确率为： 0.8048780487804879
{'activation': 'tanh', 'alpha': 0.003239806100131822, 'hidden_layer_sizes': 78, 'random_state': 42, 'solver': 'adam'}
第9884次，测试集正确率为： 0.7716186252771619
{'activation': 'tanh', 'alpha': 0.0039808947304238625, 'hidden_layer_sizes': 98, 'random_state': 42, 'solver': 'adam'}
第9885次，测试集正确率为： 0.6718403547671841
{'activation': 'tanh', 'alpha': 0.003280076000437868, 'hidden_layer_sizes': 82, 'random_state': 42, 'solver': 'adam'}
第9886次，测试集正确率为： 0.7849223946784922
{'activation': 'tanh', 'alpha': 0.0033175670231352598, 'hidden_layer_sizes': 65, 'random_state': 42, 'solver': 'adam'}
第9887次，测试集正确率为： 0.5809312638580931
{'activation': 'tanh', 'alpha': 0.003438771

第9935次，测试集正确率为： 0.8203991130820399
{'activation': 'tanh', 'alpha': 0.0034525636636946046, 'hidden_layer_sizes': 62, 'random_state': 42, 'solver': 'adam'}
第9936次，测试集正确率为： 0.7516629711751663
{'activation': 'tanh', 'alpha': 0.0037185185570278046, 'hidden_layer_sizes': 38, 'random_state': 42, 'solver': 'adam'}
第9937次，测试集正确率为： 0.7738359201773836
{'activation': 'tanh', 'alpha': 0.0036314494658973044, 'hidden_layer_sizes': 37, 'random_state': 42, 'solver': 'lbfgs'}
第9938次，测试集正确率为： 0.738359201773836
{'activation': 'tanh', 'alpha': 0.003626913939348426, 'hidden_layer_sizes': 40, 'random_state': 42, 'solver': 'adam'}
第9939次，测试集正确率为： 0.7893569844789357
{'activation': 'tanh', 'alpha': 0.0031373798688393716, 'hidden_layer_sizes': 83, 'random_state': 42, 'solver': 'adam'}
第9940次，测试集正确率为： 0.8203991130820399
{'activation': 'tanh', 'alpha': 0.003162041763247787, 'hidden_layer_sizes': 53, 'random_state': 42, 'solver': 'adam'}
第9941次，测试集正确率为： 0.7694013303769401
{'activation': 'tanh', 'alpha': 0.003212574

第9989次，测试集正确率为： 0.7073170731707317
{'activation': 'tanh', 'alpha': 0.0036246780247837875, 'hidden_layer_sizes': 43, 'random_state': 42, 'solver': 'adam'}
第9990次，测试集正确率为： 0.7804878048780488
{'activation': 'tanh', 'alpha': 0.003500453015291053, 'hidden_layer_sizes': 60, 'random_state': 42, 'solver': 'adam'}
第9991次，测试集正确率为： 0.7960088691796009
{'activation': 'logistic', 'alpha': 0.00014424431825151723, 'hidden_layer_sizes': 97, 'random_state': 42, 'solver': 'adam'}
第9992次，测试集正确率为： 0.6585365853658537
{'activation': 'tanh', 'alpha': 0.00364084666457158, 'hidden_layer_sizes': 83, 'random_state': 42, 'solver': 'adam'}
第9993次，测试集正确率为： 0.8203991130820399
{'activation': 'tanh', 'alpha': 0.0037019033228248323, 'hidden_layer_sizes': 63, 'random_state': 42, 'solver': 'adam'}
第9994次，测试集正确率为： 0.7649667405764967
{'activation': 'tanh', 'alpha': 0.0036208726950586246, 'hidden_layer_sizes': 74, 'random_state': 42, 'solver': 'lbfgs'}
第9995次，测试集正确率为： 0.729490022172949
{'activation': 'tanh', 'alpha': 0.00344